In [2]:
# Import relevant libraries
from scipy import stats
import scipy as sp
import pandas as pd
import numpy as np
import random

%matplotlib inline
import matplotlib 
import matplotlib.pyplot as plt

# Set seed for any random operations
random.seed(1)

In [4]:
# Read in train dataset from hazard model pre-processing because we want the duration instead of gross approval date
# (TODO: replace with final train data.)
train_data = pd.read_csv("test.csv")
print "Number of train points:", train_data.shape[0]

# Only want defaulted loans
train_data = train_data[train_data['Default?'] == 1]
print "Number of defaulted train points:", train_data.shape[0]

train_data.head()

Number of train points: 5481
Number of defaulted train points: 902


,Unnamed: 0,BorrName,BorrStreet,BorrCity,BorrState,BorrZip,CDC_Name,CDC_Street,CDC_City,CDC_State,...,ProjectState,BusinessType,LoanStatus,ChargeOffDate,GrossChargeOffAmount,Default?,ThirdPartyDollars_Norm,GrossApproval_Norm,TermInMonths_Norm,GrossChargeOffAmount_Norm
0,53188,RIDGEWOOD DAY SCHOOL,6606 AVONDALE,FORT COLLINS,CO,80525,"Colorado Lending Source, Ltd.",518 17th Street,Denver,CO,...,CO,CORPORATION,CHGOFF,2007-08-01 00:00:00,18174.0,True,-0.628242,0.114302,0.231833,-0.211499
10,74550,MARK J MILLER,"138-140 HIGH STREET, UNIT 140",NEWBURY,MA,MISSING,Granite State Economic Develop,One Cate Street,Portsmouth,NH,...,MA,CORPORATION,CHGOFF,2011-08-19 00:00:00,161929.0,True,-1.162767,-0.444405,0.231833,0.429574
11,31003,"SPARTAN HOMES, INC",945 NORTH 2000 WEST,SPRINGVILLE,UT,84663,Mountain West Small Business F,2595 East 3300 South,Salt Lake City,UT,...,UT,CORPORATION,CHGOFF,2002-09-24 00:00:00,0.0,True,0.000000,-0.769098,0.231833,-0.292546
27,82544,TERRADATA INC,522 SOUTH PICO AVENUE,FALLBROOK,CA,92028,CDC Small Business Finance Cor,2448 Historic Decatur,San Diego,CA,...,CA,INDIVIDUAL,CHGOFF,2011-02-18 00:00:00,166265.0,True,-1.347319,-0.760323,0.231833,0.448910
29,104476,"TYCO HOMES, INC.",301 ROUTE 940,MOUNT POCONO,PA,18344,Northeastern Economic Developm,4000-4th Street,Moosic,PA,...,PA,CORPORATION,CHGOFF,2012-02-09 00:00:00,305224.0,True,-0.887065,-0.263044,0.231833,1.068595


In [8]:
# create a new column for the loss ratio
train_data['LossRatio'] = np.minimum(train_data.GrossChargeOffAmount.values/train_data.GrossApproval.values,1.)
train_data = train_data.drop(columns=["GrossChargeOffAmount"])

print train_data['LossRatio'][:10]

0     0.037942
10    0.562253
11    0.000000
27    0.923694
29    0.872069
68    0.975756
72    0.000000
76    0.719184
82    0.884722
88    0.924806
Name: LossRatio, dtype: float64


In [ ]:
from pandas.core import datetools
import statsmodels.api as sm

#  Split the explanatory variables and the variable we want to predict
X = train_data.drop(columns=['LossRatio'])
Y = train_data['LossRatio']

# Fit to a baseline linear regression model
model = sm.OLS(Y, X).fit()

# Print out the statistics
model.summary()

# Make predictions
predictions = model.predict(X)

In [ ]:
# LINK: https://towardsdatascience.com/simple-and-multiple-linear-regression-in-python-c928425168f9

In [ ]:
# NOTE: When this is a good model, move this code into the other loss notebook in order to run 
# everything to completion